# Stock analysis

Imports and set magics:

In [1]:
import pandas as pd 
import pandas_datareader
import datetime
import matplotlib.pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

ModuleNotFoundError: No module named 'pandas'

# Read and clean data

In [2]:
**Importing the stock data from IEX to analyse the FAANG stocks in relation to the SP500**:


SyntaxError: invalid syntax (<ipython-input-2-79ebb6476faf>, line 1)

In [3]:
#Import stok data from IEX
start = datetime.datetime(2014,1,1)
end   = datetime.datetime(2019,1,1)

firms = [] 
for i,stock_name in enumerate(['FB','AAPL', 'GOOGL', 'NFLX', 'AMZN', 'SPY']): 
    firm_stock = pandas_datareader.iex.daily.IEXDailyReader(stock_name, start, end).read()
    firm_stock['firm'] = stock_name
    firms.append(firm_stock)

stocks = pd.concat(firms)

# Convert index from type 'O' to 'datetime' 
stocks.index = pd.to_datetime(stocks.index)

# Mean Closing Price Calculation 
stocks.groupby('firm').mean()

NameError: name 'datetime' is not defined

Illustration of the data

In [4]:
#Plot Raw Data - absolute growth for each stock
fig,ax = plt.subplots()
stocks.groupby('firm')['close'].plot(legend=True);
ax.set_ylabel('Price USD');
ax.set_title('Figure 1: Closing price 2014-2019');

#Plot Demeaned Data
stocks3 = stocks.copy()

# Split - Apply - Combine (de-meaning)
stocks3['close_demeaned'] = stocks3.groupby('firm')['close'].transform(lambda x: x - x.mean())

# Plot
stocks3.groupby('firm')['close_demeaned'].plot(legend=True);
plt.title('Figure 2: Demeaned stock prices');

NameError: name 'plt' is not defined

**Calculating first differences and standard deviations**:

In [5]:
#Using df.groupby('firm').var.diff(1) to calculate the first differences of closing prices. 
stocks['diff_close'] = stocks.groupby('firm').close.diff(1)
stocks.groupby('firm')['diff_close'].plot(legend=True);
plt.title('Figure 3: First-diff of the close value of each stock');

#Calculating the standard deviation for each stock
print(stocks3.groupby('firm')['close'].std())

NameError: name 'stocks' is not defined

## Analysis

**Sorting data for closing prices on all selected stocks**:

In [6]:
#Find first and last closing price for all stocks
print(firms)

#Use first and last observation of each stock to calculate growth rate over the period. 

# Growth rate 2014-2019 Facebook
x_first = 62.62
x_last = 131.09
growth_rate_FB = (x_last - x_first)/x_first * 100
print(growth_rate_FB)

# Growth rate 2014-2019 Apple
x_first = 71.1359
x_last = 157.0663
growth_rate_AAPL = (x_last - x_first)/x_first * 100
print(growth_rate_AAPL)

# Growth rate 2014-2019 Google
x_first = 568.0124
x_last = 1044.9600
growth_rate_GOOGL = (x_last - x_first)/x_first * 100
print(growth_rate_GOOGL)


# Growth rate 2014-2019 Amazon
x_first = 342.990
x_last = 1501.970
growth_rate_AMZN = (x_last - x_first)/x_first * 100
print(growth_rate_AMZN)

# Growth rate 2014-2019 Netflix
x_first = 52.0985
x_last = 267.66
growth_rate_NFLX = (x_last - x_first)/x_first * 100
print(growth_rate_NFLX)

# Growth rate 2014-2019 SP500
x_first = 170.4958
x_last = 248.8239
growth_rate_SPY = (x_last - x_first)/x_first * 100
print(growth_rate_SPY)

NameError: name 'firms' is not defined

## Explore data set

In order to be able to **explore the raw data**, we here provide an **interactive plot** of the growth rates

In [7]:
#def plot_empl_inc(empl,inc,dataset,municipality): 
    
 # if dataset == 'Employment':
       # df = empl
      #  y = 'employment'
   # else:
     #   df = inc
       # y = 'income'
    
    #I = df['municipality'] == municipality
    #ax = df.loc[I,:].plot(x='year', y=y, style='-o')

The **interactive plot** is:

In [8]:
#widgets.interact(plot_empl_inc, 
    
 #   empl = widgets.fixed(empl_long),
  #  inc = widgets.fixed(inc_long),
   # dataset = widgets.Dropdown(description='Dataset', 
    #                           options=['Employment','Income']),
    #municipality = widgets.Dropdown(description='Municipality', 
     #                               options=empl_long.municipality.unique())
                 
#); 

#Plot the growth rates against each other 

import numpy as np
import matplotlib.pyplot as plt
 
# data to plot
n_groups = 6
growth_rate = (growth_rate_SPY, growth_rate_NFLX, growth_rate_AAPL, growth_rate_GOOGL, growth_rate_FB, growth_rate_AMZN)
 
# create plot
fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35
opacity = 0.8
 
rects1 = plt.bar(index, growth_rate, bar_width,
alpha=opacity,
color='m',
label='Growth Rate')
 
plt.xlabel('Firm')
plt.ylabel('Percentage')
plt.title('Figure 4: Growth Rate 2014 - 2019')
plt.xticks(index + bar_width, ('SPY', 'FB', 'AAPL', 'AMZN' 'NFLX','GOOGL'))
plt.legend()
 
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'numpy'

ADD SOMETHING HERE IF THE READER SHOULD KNOW THAT E.G. SOME MUNICIPALITY IS SPECIAL.

# Conclusion

ADD CONCISE CONLUSION.